the following code is ok to use

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# load data
nyc_boroughs = gpd.read_file('NYC_Borough_Boundary.geojson')
joined_data = pd.read_csv("NewYork_add_all.csv")

# transfer GeoDataFrame
# geometry = [Point(xy) for xy in zip(house_data['LONGITUDE'], house_data['LATITUDE'])]
# house_data_gdf = gpd.GeoDataFrame(house_data, geometry=geometry, crs="EPSG:4326")

# # join boroname
# joined_data = gpd.sjoin(house_data_gdf, nyc_boroughs, how="inner", predicate='intersects')

# calculate average price
average_prices = joined_data.groupby('BoroName')['PRICE'].mean().reset_index()
average_prices.columns = ['BoroName', 'AveragePrice']

# merge to nyc_boroughs_with_prices
nyc_boroughs_with_prices = nyc_boroughs.merge(average_prices, on='BoroName')
nyc_boroughs_with_prices['AveragePriceInMillions'] = nyc_boroughs_with_prices['AveragePrice'] / 1000000
nyc_boroughs_with_prices['AveragePriceInMillions'] = nyc_boroughs_with_prices['AveragePriceInMillions'].round(2)


# group by both boro and type
grouped_data = joined_data.groupby(['BoroName', 'TYPE_VIZ'])['PRICE'].agg(['mean', 'count']).reset_index()
grouped_data.columns = ['BoroName', 'TYPE_VIZ', 'AveragePrice', 'Count']


In [2]:
broker_count = joined_data.groupby(['BoroName', 'BROKERTITLE']).size().reset_index(name='Transactions')
top_brokers_per_boro = broker_count.groupby('BoroName').apply(lambda x: x.sort_values('Transactions', ascending=False).head(5))
top_brokers_per_boro['BROKERTITLE'] = top_brokers_per_boro['BROKERTITLE'].str.replace("Brokered by ", "", regex=False)


# Reset the index to make the DataFrame tidy
top_brokers_per_boro.reset_index(drop=True, inplace=True)
top_brokers_per_boro.head(20)


top_brokers_nyc = joined_data['BROKERTITLE'].value_counts().head(5).reset_index()
top_brokers_nyc.columns = ['BROKERTITLE', 'Transactions']

top_brokers_nyc['BROKERTITLE'] = top_brokers_nyc['BROKERTITLE'].str.replace("Brokered by ", "", regex=False)


top_brokers_nyc['BoroName'] = 'All'

top_brokers_combined = pd.concat([top_brokers_per_boro, top_brokers_nyc], ignore_index=True)
top_brokers_combined.reset_index(drop=True, inplace=True)

top_brokers_combined

,BoroName,BROKERTITLE,Transactions
0,Bronx,Exp Realty,26
1,Bronx,TREBACH REALTY INC,20
2,Bronx,Re/Max In The City,18
3,Bronx,BESMATCH REAL ESTATE,17
4,Bronx,RE MAX Distinguished Homes & Properties,16
5,Brooklyn,COMPASS,109
6,Brooklyn,RE MAX Edge,64
7,Brooklyn,RE MAX Real Estate Professionals,43
8,Brooklyn,Corcoran Park Slope,34
9,Brooklyn,Corcoran Brooklyn Heights,30


In [3]:
overall_broker = top_brokers_combined[top_brokers_combined["BoroName"]=="All"]
overall_broker

,BoroName,BROKERTITLE,Transactions
25,All,COMPASS,456
26,All,Douglas Elliman - 575 Madison Ave,110
27,All,Brown Harris Stevens,93
28,All,Corcoran East Side,91
29,All,RE MAX Edge,79


In [4]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Initialize Dash app
app = dash.Dash(__name__)


app.layout = html.Div([
    # Left side of the screen
    html.Div([
        # Map graph
        dcc.Graph(id='nyc-map', style={'height': '56vh'}),

        # Filter section
        html.Label('Number of Bedrooms:'),
        dcc.Dropdown(
        id='num-bedrooms-dropdown',
        options=[
        {'label': '1', 'value': 1},
        {'label': '2', 'value': 2},
        {'label': '3', 'value': 3},
        {'label': '4', 'value': 4},
        {'label': '5+', 'value': '5+'} 
        ],
        value=1  
        ),

        # Dropdown for number of bathrooms
        html.Label('Number of Bathrooms:'),
        dcc.Dropdown(
        id='num-bathrooms-dropdown',
        options=[
        {'label': '1', 'value': 1},
        {'label': '2', 'value': 2},
        {'label': '3', 'value': 3},
        {'label': '4', 'value': 4},
        {'label': '5+', 'value': '5+'}  
        ],
        value=1  
        ),
    # Div to display the price range
    html.Div(id='price-range-display', children='Select an area and set filters to see price range'),
    ], style={'width': '48%', 'display': 'inline-block'}),

    html.Div([
        dcc.Graph(id='bubble-chart', style={'height': '50vh'}),
        html.Div(style={'height': '20px'}),  # Add a spacer div
        dcc.Graph(id='broker-histogram', style={'height': '40vh', 'width':'60vh'}),
    ], style={'width': '32%', 'margin': '8px', 'boxShadow': '2px 2px 5px grey', 'display': 'flex', 'flexDirection': 'column', 'justifyContent': 'flex-start'})
], style={'display': 'flex', 'flexDirection': 'row', 'alignItems': 'start', 'height': '100vh', 'backgroundColor': '#f0f0f0'})



# Define callback function to update map and bubble chart
@app.callback(
    [Output('nyc-map', 'figure'),
     Output('bubble-chart', 'figure'),
     Output('broker-histogram', 'figure'),
     Output('price-range-display', 'children')],  # Output for the price range display
    [Input('nyc-map', 'clickData'),
     Input('num-bedrooms-dropdown', 'value'),
     Input('num-bathrooms-dropdown', 'value')]
)
def update_content(clickData, num_bedrooms, num_bathrooms):
    price_range_text = 'Select an area and set filters to see price range'
    # Placeholder for updated opacities
    opacities = [0.5 if not clickData else 0.2 for _ in nyc_boroughs_with_prices.index]
    
    if clickData:
        clicked_boro = clickData['points'][0]['customdata'][0]
        opacities = [0.5 if boro == clicked_boro else 0.2 for boro in nyc_boroughs_with_prices['BoroName']]
        if num_bedrooms == '5+':
            filtered_data_prices = joined_data[(joined_data['BEDS'] >= 5) & 
                                               (joined_data['BoroName'] == clicked_boro)]
        else:
            filtered_data_prices = joined_data[(joined_data['BEDS'] == num_bedrooms) & 
                                               (joined_data['BoroName'] == clicked_boro)]
        

        if num_bathrooms == '5+':
            filtered_data_prices = filtered_data_prices[filtered_data_prices['BATH'] >= 5]
        else:
            filtered_data_prices = filtered_data_prices[filtered_data_prices['BATH'] == num_bathrooms]
        # Calculate the price range
        min_price = filtered_data_prices['PRICE'].min()
        max_price = filtered_data_prices['PRICE'].max()
        price_range_text = f"Price Range: ${min_price} - ${max_price}" if not filtered_data_prices.empty else "No data for selected filters"
    
    fig = px.choropleth_mapbox(nyc_boroughs_with_prices,
                               geojson=nyc_boroughs_with_prices.geometry,
                               locations=nyc_boroughs_with_prices.index,
                               color="AveragePrice",
                               color_continuous_scale=px.colors.sequential.Viridis,
                               mapbox_style="carto-positron",
                               zoom=9, center={"lat": 40.730610, "lon": -73.935242},
                               opacity=opacities,
                               custom_data=["BoroName"])
    fig.update_layout(mapbox_zoom=9, mapbox_center={"lat": 40.730610, "lon": -73.935242})
    fig.update_layout(margin={"r":3,"t":3,"l":3,"b":3})
    fig.update_layout(coloraxis_colorbar=dict(thickness=10, title_font_size=10))
    
    
    # Update bubble chart based on clickData
    # Placeholder for bubble chart logic
    if not clickData:
        overall_avg_price_count = joined_data.groupby('TYPE_VIZ')['PRICE'].agg(['mean', 'count']).reset_index()
        overall_avg_price_count.columns = ['TYPE_VIZ', 'AveragePrice', 'Count']
        bubble_fig = px.scatter(overall_avg_price_count,
                                x='TYPE_VIZ', 
                                y='AveragePrice', 
                                size='Count', 
                                color='TYPE_VIZ',
                                hover_name='TYPE_VIZ', 
                                size_max=45,
                                title="Overall Properties in NYC")
        hist_fig = px.bar(overall_broker, 
                      x='Transactions', 
                      y='BROKERTITLE', 
                      color='BoroName', 
                      title=f"Top 5 Brokers in NYC",
                      labels={'BROKERTITLE': 'Broker', 'Transactions': 'Transaction Count'})
    else:
        boro_name = clicked_boro
        filtered_data = grouped_data[grouped_data['BoroName'] == boro_name]
        bubble_fig = px.scatter(filtered_data,
                                x='TYPE_VIZ', 
                                y='AveragePrice', 
                                size='Count', 
                                color='TYPE_VIZ',
                                hover_name='TYPE_VIZ', 
                                size_max=45,
                                title=f"Properties in {boro_name}") 
        filtered_data_broker = top_brokers_per_boro[top_brokers_combined['BoroName'] == boro_name]
        hist_fig = px.bar(filtered_data_broker, 
                      x='Transactions', 
                      y='BROKERTITLE', 
                      color='BoroName', 
                      title=f"Top 5 Brokers in {boro_name}",
                      labels={'BROKERTITLE': 'Broker', 'Transactions': 'Transaction Count'})


    
    #bubble_fig = px.scatter()  # Update this with actual data
    #bubble_fig.update_layout(title_text='Property Analysis', title_font_size=16)
    bubble_fig.update_traces(
                             selector=dict(mode='markers'),showlegend=False)
    bubble_fig.update_layout(hoverlabel=dict(namelength=-1))
    bubble_fig.update_layout(margin={"r":20,"t":38,"l":3,"b":3},
                             xaxis_title='',
                             xaxis_tickangle=0, 
                             xaxis_tickfont=dict(size=11), 
                             yaxis_title_font=dict(size=15) )
    bubble_fig.update_traces(hovertemplate='Property Number: %{marker.size}<br>Average Price: %{y}', showlegend=False)

    hist_fig.update_layout(barmode='group', xaxis={'categoryorder': 'total descending'})
    hist_fig.update_layout(margin={"r":0, "t":30, "l":0, "b":0})
    hist_fig.update_layout(showlegend=False)

    return fig, bubble_fig,hist_fig,price_range_text


    
if __name__ == '__main__':
    app.run_server(debug=True, port=8052)

/var/folders/4x/hg1yh7mj1t74r8jh00bn4bfh0000gp/T/ipykernel_9362/1232866099.py:136: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

